In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Fetched 337 kB in 1s (261 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [4]:
# 2. Create a temporary view of the DataFrame.

# Read the CSV data into a DataFrame
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Create a temporary table called "home_sales"
df.createOrReplaceTempView("home_sales")

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
start_time = time.time()
query1 = spark.sql("""
    SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")
query1.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+

--- 4.356268405914307 seconds ---


In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
start_time = time.time()
query2 = spark.sql("""
    SELECT year(to_date(concat_ws('-', date_built, '01', '01'))) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year
    ORDER BY year
""")
query2.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+

--- 1.4947786331176758 seconds ---


In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
start_time = time.time()
query3 = spark.sql("""
    SELECT year(to_date(concat_ws('-', date_built, '01', '01'))) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY year
    ORDER BY year
""")
query3.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+

--- 1.0649089813232422 seconds ---


In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
query4 = spark.sql("""
    SELECT AVG(view) AS avg_view
    FROM home_sales
    WHERE price >= 350000
""")
query4.show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|  32.264|
+--------+

--- 0.6823608875274658 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
query5 = spark.sql("""
    SELECT AVG(view) AS avg_view
    FROM home_sales
    WHERE price >= 350000
""")
query5.show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|  32.264|
+--------+

--- 2.279120683670044 seconds ---


In [12]:
# 10. Read the parquet formatted data.
df.write.parquet('parquet_home_sales',mode='overwrite')
parquet_data_df = spark.read.parquet('parquet_home_sales')

In [13]:
# 11. Partition by the "date_built" field on the formatted parquet home sales data
parquet_data_df.write.partitionBy("date_built").parquet('partitioned_parquet_home_sales', mode='overwrite')

In [14]:
# 12. Create a temporary table for the parquet data.
partitioned_df = spark.read.parquet('partitioned_parquet_home_sales')
partitioned_df.createOrReplaceTempView('partitioned_home_sales')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()
query6 = spark.sql("""
    SELECT AVG(view) AS avg_view
    FROM partitioned_home_sales
    WHERE price >= 350000
""")
query6.show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|  32.264|
+--------+

--- 0.7790007591247559 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [17]:
# 15. Check if the home_sales is no longer cached
print(spark.catalog.isCached('home_sales'))

False
